## RNN Language Model

Below is a diagram of the RNN computation that we will implement below. We're plugging characters into the RNN with a 1-hot encoding and expecting it to predict the next character. In this example the training data is the string "hello", so there are 4 letters in the vocabulary: [h,e,l,o].

<img src="rnnlm.jpeg">

In [1]:
import numpy as np
np.random.seed(1337)

In [5]:
# data I/O
# get shakespeare from http://cs.stanford.edu/people/karpathy/shakespeare.txt
data = open('shakespeare.txt', 'r').read() # should be simple plain text file
chars = list(set(data))
data_size, vocab_size = len(data), len(chars)
print ('data has %d characters, %d unique.' % (data_size, vocab_size))

data has 4573337 characters, 67 unique.


In [7]:
char_to_ix = { ch:i for i,ch in enumerate(chars) }
ix_to_char = { i:ch for i,ch in enumerate(chars) }
char_to_ix['a']

32

In [9]:
# lets sample a batch of data
seq_length = 25 # number of characters in the batch
p = 220000 # point in the book to sample from
data[p:p+seq_length] # print a chunk of data

' thing when he was young,'

In [11]:
inputs = [char_to_ix[ch] for ch in data[p:p+seq_length]]
targets = [char_to_ix[ch] for ch in data[p+1:p+seq_length+1]]
print(inputs)
print(targets)

[15, 59, 27, 40, 48, 50, 15, 35, 27, 57, 48, 15, 27, 57, 15, 35, 32, 6, 15, 33, 24, 16, 48, 50, 60]
[59, 27, 40, 48, 50, 15, 35, 27, 57, 48, 15, 27, 57, 15, 35, 32, 6, 15, 33, 24, 16, 48, 50, 60, 44]


In [12]:
# lets plug the first character into the RNN
ix_input = inputs[0]
ix_target = targets[0]
# encode the input character with a 1-hot representation
x = np.zeros((vocab_size,1))
x[ix_input] = 1
print(x.ravel())

[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]


In [13]:
# create random starting parameters
hidden_size = 10
Wxh = np.random.randn(hidden_size, vocab_size)*0.01 # input to hidden
Whh = np.random.randn(hidden_size, hidden_size)*0.01 # hidden to hidden
Why = np.random.randn(vocab_size, hidden_size)*0.01 # hidden to output
bh = np.zeros((hidden_size, 1)) # hidden bias
by = np.zeros((vocab_size, 1)) # output bias

In [15]:
# compute the hidden state
h_prev = np.zeros((hidden_size, 1))
h = np.tanh(np.dot(Wxh, x) + np.dot(Whh, h_prev + bh))
print (h.ravel())

[-0.01691059  0.01554663  0.00144864 -0.00465686  0.01337068 -0.00197203
 -0.01462834  0.0021453   0.00716481 -0.00585728]


In [16]:
# compute the scores for next character
y = np.dot(Why, h) + by
y.ravel()

array([ 2.02252254e-04, -4.09443095e-04,  2.91610545e-05,  1.17896316e-04,
        2.01553857e-07, -7.65804840e-05, -2.04271279e-04, -1.33499420e-04,
       -3.76494116e-04, -7.20562504e-04,  2.02576371e-04, -1.41213486e-04,
       -6.78915199e-04,  5.91397032e-04,  1.41375269e-04,  6.63116525e-05,
        2.54640037e-04,  1.71579179e-04, -2.75887670e-04, -7.94141193e-05,
       -6.63724086e-04,  5.33985243e-04, -4.56874570e-04, -5.84645905e-04,
       -3.99740338e-04, -1.84093920e-04,  3.34857922e-04, -2.02078350e-04,
       -3.56369422e-04,  4.39302494e-04, -6.87915834e-04, -1.57101331e-04,
       -5.85135824e-05, -2.73254714e-04,  3.95023049e-04,  1.91172537e-04,
        1.86934623e-04,  5.44955791e-04, -3.00361815e-04,  2.30560301e-04,
       -3.26725402e-04,  3.43734988e-04, -3.36650132e-04,  3.54415612e-04,
       -4.64958818e-05,  5.58317235e-04,  1.51545532e-04, -4.97279834e-04,
        1.91349645e-04, -1.79453959e-04,  4.30439637e-04,  3.97501704e-05,
        6.01827494e-06, -

In [18]:
# the scores are unnormalized log probabilities. compute the probabilities
p = np.exp(y) / np.sum(np.exp(y))
print(p.ravel())
print('probabilities sum to ', p.sum())

[0.01492899 0.01491986 0.01492641 0.01492773 0.01492598 0.01492483
 0.01492292 0.01492398 0.01492035 0.01491522 0.014929   0.01492386
 0.01491584 0.0149348  0.01492808 0.01492696 0.01492977 0.01492853
 0.01492186 0.01492479 0.01491607 0.01493394 0.01491915 0.01491725
 0.01492001 0.01492323 0.01493097 0.01492296 0.01492065 0.01493253
 0.01491571 0.01492363 0.0149251  0.01492189 0.01493187 0.01492883
 0.01492876 0.01493411 0.01492149 0.01492941 0.0149211  0.0149311
 0.01492095 0.01493126 0.01492528 0.01493431 0.01492823 0.01491855
 0.01492883 0.01492329 0.0149324  0.01492657 0.01492606 0.0149166
 0.01492227 0.01492342 0.0149223  0.01492951 0.01492411 0.01492727
 0.01492919 0.01492284 0.014928   0.01492609 0.01491772 0.01493093
 0.01493449]
probabilities sum to  1.0


In [19]:
print('probability assigned to the correct next character is right now: ', p[ix_target,0])

probability assigned to the correct next character is right now:  0.01492726675670666


In [20]:
loss = -np.log(p[ix_target,0])
print('the cross-entropy (softmax) loss is ', loss)

the cross-entropy (softmax) loss is  4.20456575473928


In [21]:
# compute the gradient on y
dy = np.copy(p)
dy[ix_target] -= 1
print(dy.ravel())
print('sum of dy is ', dy.sum())
print('the gradient for the correct character (%s) is: %s' % (ix_to_char[ix_target], dy[ix_target,0]))
print ('the gradient for the character (a) is: ', dy[char_to_ix['a'],0])

[ 0.01492899  0.01491986  0.01492641  0.01492773  0.01492598  0.01492483
  0.01492292  0.01492398  0.01492035  0.01491522  0.014929    0.01492386
  0.01491584  0.0149348   0.01492808  0.01492696  0.01492977  0.01492853
  0.01492186  0.01492479  0.01491607  0.01493394  0.01491915  0.01491725
  0.01492001  0.01492323  0.01493097  0.01492296  0.01492065  0.01493253
  0.01491571  0.01492363  0.0149251   0.01492189  0.01493187  0.01492883
  0.01492876  0.01493411  0.01492149  0.01492941  0.0149211   0.0149311
  0.01492095  0.01493126  0.01492528  0.01493431  0.01492823  0.01491855
  0.01492883  0.01492329  0.0149324   0.01492657  0.01492606  0.0149166
  0.01492227  0.01492342  0.0149223   0.01492951  0.01492411 -0.98507273
  0.01492919  0.01492284  0.014928    0.01492609  0.01491772  0.01493093
  0.01493449]
sum of dy is  -1.0408340855860843e-17
the gradient for the correct character (t) is: -0.9850727332432934
the gradient for the character (a) is:  0.014925099219633975


In [22]:
# we computed [y = np.dot(Why, h) + by]; Backpropagate to Why, h, and by
dWhy = np.dot(dy, h.T)
dh = np.dot(Why.T, dy)
dby = np.copy(dy)
print ('the hidden vector activations were:')
print (h.ravel())
print ('the gradients are:')
print (dh.ravel())
print ('the gradients dWhy have size: ', dWhy.shape)
print ('a small sample is:')
print (dWhy[:4,:4])

the hidden vector activations were:
[-0.01691059  0.01554663  0.00144864 -0.00465686  0.01337068 -0.00197203
 -0.01462834  0.0021453   0.00716481 -0.00585728]
the gradients are:
[ 0.00559956  0.00192929  0.00399658  0.01773859  0.00816384  0.01092136
 -0.00496552 -0.01166842 -0.02226151 -0.00665568]
the gradients dWhy have size:  (67, 10)
a small sample is:
[[-2.52458048e-04  2.32095461e-04  2.16267486e-05 -6.95222854e-05]
 [-2.52303667e-04  2.31953532e-04  2.16135237e-05 -6.94797720e-05]
 [-2.52414353e-04  2.32055291e-04  2.16230056e-05 -6.95102528e-05]
 [-2.52436752e-04  2.32075883e-04  2.16249244e-05 -6.95164211e-05]]


In [23]:
# we computed [h = np.tanh(np.dot(Wxh, x) + np.dot(Whh, h_prev + bh))]; 
# Backprop into Wxh, x, Whh, h_prev, bh:
dh_before_tanh = (1-h**2)*dh
dbh = np.copy(dh_before_tanh)
dWxh = np.dot(dh_before_tanh, x.T)
dWhh = np.dot(dh_before_tanh, h.T)
dh_prev = np.dot(Whh.T, dh_before_tanh)
print ('small sample of Whh:')
print (Whh[:4,:4])

small sample of Whh:
[[-0.0028736  -0.00195895  0.00885911 -0.00349354]
 [-0.00773272 -0.00051873  0.00219991 -0.00234756]
 [ 0.01687054 -0.01221995  0.00125455 -0.00568523]
 [-0.00031645  0.00514377  0.01194564  0.0070584 ]]


In [24]:
# we now have the gradients for all parameters! (Wxh, Whh, Why, bh, by)
# lets do a parameter update
learning_rate = 0.1
Wxh2 = Wxh - learning_rate * dWxh
Whh2 = Whh - learning_rate * dWhh
Why2 = Why - learning_rate * dWhy
bh2 = bh - learning_rate * dbh
by2 = by - learning_rate * dby

In [25]:
# these parameters should be much better! lets try it out:
h2 = np.tanh(np.dot(Wxh2, x) + np.dot(Whh2, h_prev + bh2))
y2 = np.dot(Why2, h2) + by2
p2 = np.exp(y2) / np.sum(np.exp(y2))
print ('probability assigned to the correct next character was: ', p[ix_target,0])
print ('probability assigned to the correct next character is now: ', p2[ix_target,0])
loss2 = -np.log(p2[ix_target,0])
print( 'the cross-entropy (softmax) loss was ', loss)
print ('the loss is now ', loss2)

probability assigned to the correct next character was:  0.01492726675670666
probability assigned to the correct next character is now:  0.01647507098136339
the cross-entropy (softmax) loss was  4.20456575473928
the loss is now  4.105906890174113


In [19]:
# note: the probability for the correct character went up! (and the loss went down)

In [26]:
# putting it together with loops
def lossFun(inputs, targets, hprev):
    """
    inputs,targets are both list of integers.
    hprev is Hx1 array of initial hidden state
    returns the loss, gradients on model parameters, and last hidden state
    """
    xs, hs, ys, ps = {}, {}, {}, {}
    hs[-1] = np.copy(hprev)
    loss = 0
    
    # forward pass
    for t in range(len(inputs)):
        xs[t] = np.zeros((vocab_size,1)) # encode in 1-of-k representation
        xs[t][inputs[t]] = 1
        hs[t] = np.tanh(np.dot(Wxh, xs[t]) + np.dot(Whh, hs[t-1]) + bh) # hidden state
        ys[t] = np.dot(Why, hs[t]) + by # unnormalized log probabilities for next chars
        ps[t] = np.exp(ys[t]) / np.sum(np.exp(ys[t])) # probabilities for next chars
        loss += -np.log(ps[t][targets[t],0]) # softmax (cross-entropy loss)
    
    # backward pass: compute gradients going backwards
    dWxh, dWhh, dWhy = np.zeros_like(Wxh), np.zeros_like(Whh), np.zeros_like(Why)
    dbh, dby = np.zeros_like(bh), np.zeros_like(by)
    dhnext = np.zeros_like(hs[0])
    for t in reversed(range(len(inputs))):
        dy = np.copy(ps[t])
        dy[targets[t]] -= 1 # backprop into y
        dWhy += np.dot(dy, hs[t].T)
        dby += dy
        dh = np.dot(Why.T, dy) + dhnext # backprop into h
        dhraw = (1 - hs[t] * hs[t]) * dh # backprop through tanh nonlinearity
        dbh += dhraw
        dWxh += np.dot(dhraw, xs[t].T)
        dWhh += np.dot(dhraw, hs[t-1].T)
        dhnext = np.dot(Whh.T, dhraw)
        
    # clip to mitigate exploding gradients
    for dparam in [dWxh, dWhh, dWhy, dbh, dby]:
        np.clip(dparam, -5, 5, out=dparam)
    
    return loss, dWxh, dWhh, dWhy, dbh, dby, hs[len(inputs)-1]

In [27]:
loss, dWxh, dWhh, dWhy, dbh, dby, hnew = lossFun(inputs, targets, h_prev)
print (loss)

105.11784328521104


In [28]:
# TODO: write the sampling code
def sample(h, seed_ix, n):
    """ 
    sample a sequence of integers from the model 
    h is initial memory state, seed_ix is seed letter for first time step
    n is the number of time steps to sample for
    """
    x = np.zeros((vocab_size, 1))
    x[seed_ix] = 1
    ixes = [] # sampled indices
    for t in range(n):
        pass # TODO: run the RNN for one time step, sample from distribution
    return ixes


In [23]:
# TODO: write the optimization loop
# Loop over the dataset from beginning to end, sampling batches of characters seq_length long
# Call the loss function and get the gradients
# Perform a parameter update
# Sample some examples from the model

In [29]:
# Stochastic Gradient Descent
n, p = 0, 0
smooth_loss = -np.log(1.0/vocab_size)*seq_length # loss at iteration 0
learning_rate = 1e-3
while n < 10000:
    # prepare inputs (we're sweeping from left to right in steps seq_length long)
    if p+seq_length+1 >= len(data) or n == 0: 
        hprev = np.zeros((hidden_size,1)) # reset RNN memory
        p = 0 # go from start of data
    inputs = [char_to_ix[ch] for ch in data[p:p+seq_length]]
    targets = [char_to_ix[ch] for ch in data[p+1:p+seq_length+1]]

    # forward seq_length characters through the net and fetch gradient
    loss, dWxh, dWhh, dWhy, dbh, dby, hprev = lossFun(inputs, targets, hprev)
    smooth_loss = smooth_loss * 0.999 + loss * 0.001
    if n % 100 == 0: print ('iter %d, loss: %f' % (n, smooth_loss)) # print progress

    # perform parameter update with Adagrad
    for param, dparam in zip([Wxh, Whh, Why, bh, by], 
                                [dWxh, dWhh, dWhy, dbh, dby]):
        param += -learning_rate * dparam

    p += seq_length # move data pointer
    n += 1 # iteration counter 

iter 0, loss: 105.117315
iter 100, loss: 104.983428
iter 200, loss: 104.623506
iter 300, loss: 104.093932
iter 400, loss: 103.408087
iter 500, loss: 102.742825
iter 600, loss: 101.887765
iter 700, loss: 101.019030
iter 800, loss: 100.226599
iter 900, loss: 99.278336
iter 1000, loss: 98.311200
iter 1100, loss: 97.269222
iter 1200, loss: 96.135463
iter 1300, loss: 94.971205
iter 1400, loss: 94.009555
iter 1500, loss: 93.148438
iter 1600, loss: 92.125801
iter 1700, loss: 91.396635
iter 1800, loss: 90.822384
iter 1900, loss: 89.993188
iter 2000, loss: 88.971474
iter 2100, loss: 88.298134
iter 2200, loss: 87.808791
iter 2300, loss: 87.190983
iter 2400, loss: 86.830461
iter 2500, loss: 86.598521
iter 2600, loss: 86.067505
iter 2700, loss: 85.818215
iter 2800, loss: 85.895126
iter 2900, loss: 85.631623
iter 3000, loss: 85.174037
iter 3100, loss: 85.296305
iter 3200, loss: 85.270654
iter 3300, loss: 85.047483
iter 3400, loss: 84.906948
iter 3500, loss: 84.596379
iter 3600, loss: 84.436224
iter

In [31]:
def sample(h, seed_ix, n):
    """ 
    sample a sequence of integers from the model 
    h is memory state, seed_ix is seed letter for first time step
    """
    x = np.zeros((vocab_size, 1))
    x[seed_ix] = 1
    ixes = []
    for t in range(n):
        h = np.tanh(np.dot(Wxh, x) + np.dot(Whh, h) + bh)
        y = np.dot(Why, h) + by
        p = np.exp(y) / np.sum(np.exp(y))
        ix = np.random.choice(range(vocab_size), p=p.ravel())
        x = np.zeros((vocab_size, 1))
        x[ix] = 1
        ixes.append(ix)
    return ixes

sample_ix = sample(hprev, char_to_ix['a'], 1000)
txt = ''.join(ix_to_char[ix] for ix in sample_ix)
print (txt)

aunmtinsrl:ne dew t:V m Au:rht Svonresim,sp
e om reoYI ermew bsr yoserlgyte whet yd
e  rtofot
u.rlbt lilEbutSeomfna sOtimb UeN nedeye o, hlsnhAnnm ireD lt HsGkAsaIgeeoA,de a:
icnyhnd, o i lO aI rlnJeg nhlmr-s ,ngO 'etXabnroneT tElcito
ui pnaa ,n y tnrshs:e TdaEheot  ipuas ynon:o sslo
e m dtnyA
wfhksrrib
eu
r  t h herrecluhacFi t tlpaueem it Hsd aertn
 lie orine lod d AowredI thaaenit 
CNwso mIhizben.:dsIaum a?e:UrrP vn e ld a Sglah desh to eh 
 tia ulpauie mvoiesre os 
t milhwr wt aggooali
hTdei t&iit, ndyn mGlhLen
 ry ohdse
 o
lw terKn mgwtwTne io eo casrdarCyh n d n wP, ee eo e r.ed nKiN?dtmda:A
e anralrcqwant obtie g
rdeal
l oYn itqsl r re Csr rsto nt atnoldhO]yrdi , Ishh htgo o yOsliilmoogoeMhae lheaennh etsy n tabe yl]k ehieed;:
sefperasagrse.rmp
oantfrLE, unanne sseatenrs:c  wo tyh uhn te mthh  hy ileihye' e ed .ev t Bc hiA ta y und ureh res rey 
muavmno af ufeaolidAkeratisEn iMit bdg;KeltIlnd sCCtanpTH l tGakh sLraeriohradbene 
' le mit,t td sancp edyd ndI ktrnolar hrsNWe
au  oi